In [362]:
import nltk
from os import getcwd
import torch.nn as nn
import torch.nn.functional as F
import torch
import torch.optim as optim

In [363]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [364]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples 

from utils import process_tweet, build_freqs

In [365]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [366]:
# split the data into two pieces, one for training and one for testing (validation set) 
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg 
test_x = test_pos + test_neg

In [367]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [368]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [369]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11346


### Process tweets

In [370]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [371]:
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def extract_features(tweet, freqs):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)
    
    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3)) 
    
    #bias term is set to 1
    x[0,0] = 1 
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # loop through each word in the list of words
    for word in word_l:
        
        # increment the word count for the positive label 1
        positive = freqs.get((word,1))
        if(positive==None):
            positive=0
            
        negative = freqs.get((word,0))
        if(negative==None):
            negative=0

        
            
        
        x[0,1] += positive
        
        # increment the word count for the negative label 0
        x[0,2] += negative
        
    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x

In [372]:
tmp1 = extract_features(train_x[0], freqs)
print(tmp1)

[[1.00e+00 3.02e+03 6.10e+01]]


In [389]:



class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(3, 1)
 

    def forward(self, x):
        x = torch.sigmoid(self.fc1(x))
        return x



net = Net()
print(net)

Net(
  (fc1): Linear(in_features=3, out_features=1, bias=True)
)


In [390]:
net.state_dict()["fc1.weight"].data.copy_(torch.tensor([[0.0,  0.0,  0.0]]))  
net.state_dict()["fc1.bias"].data.copy_(torch.tensor([0.0])) 


tensor([0.])

In [391]:
print("Model's state_dict:")
for param_tensor in net.state_dict():
    print(param_tensor, "\t", net.state_dict()[param_tensor])

Model's state_dict:
fc1.weight 	 tensor([[0., 0., 0.]])
fc1.bias 	 tensor([0.])


In [392]:
def my_loss(h, y):
    h = h.detach().numpy()
    y = y.detach().numpy()
    m = (h.shape)[0]
    J = -1/m * ((np.dot(np.transpose(y),np.log(h)))+ (np.dot(np.transpose(1-y),np.log(1-h))))
    #print("J",type(J))
   # J = np.ndarray(J)
    J = torch.from_numpy(J)
    J.requires_grad = True
    #print("J",J)
    return J



In [393]:
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)
Y = train_y
X = torch.from_numpy(X).float()
Y = torch.from_numpy(Y).float()

In [394]:
criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=1e-9, momentum=0.9)

In [395]:
for i in range(1500):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs, Y)
    loss.backward()
    optimizer.step()
    print("Loss",loss)

Loss tensor(0.2500, grad_fn=<MseLossBackward>)
Loss tensor(0.2497, grad_fn=<MseLossBackward>)
Loss tensor(0.2492, grad_fn=<MseLossBackward>)
Loss tensor(0.2485, grad_fn=<MseLossBackward>)
Loss tensor(0.2476, grad_fn=<MseLossBackward>)
Loss tensor(0.2465, grad_fn=<MseLossBackward>)
Loss tensor(0.2453, grad_fn=<MseLossBackward>)
Loss tensor(0.2439, grad_fn=<MseLossBackward>)
Loss tensor(0.2424, grad_fn=<MseLossBackward>)
Loss tensor(0.2408, grad_fn=<MseLossBackward>)
Loss tensor(0.2392, grad_fn=<MseLossBackward>)
Loss tensor(0.2374, grad_fn=<MseLossBackward>)
Loss tensor(0.2356, grad_fn=<MseLossBackward>)
Loss tensor(0.2337, grad_fn=<MseLossBackward>)
Loss tensor(0.2318, grad_fn=<MseLossBackward>)
Loss tensor(0.2299, grad_fn=<MseLossBackward>)
Loss tensor(0.2279, grad_fn=<MseLossBackward>)
Loss tensor(0.2259, grad_fn=<MseLossBackward>)
Loss tensor(0.2239, grad_fn=<MseLossBackward>)
Loss tensor(0.2219, grad_fn=<MseLossBackward>)
Loss tensor(0.2199, grad_fn=<MseLossBackward>)
Loss tensor(0

Loss tensor(0.0861, grad_fn=<MseLossBackward>)
Loss tensor(0.0859, grad_fn=<MseLossBackward>)
Loss tensor(0.0856, grad_fn=<MseLossBackward>)
Loss tensor(0.0854, grad_fn=<MseLossBackward>)
Loss tensor(0.0851, grad_fn=<MseLossBackward>)
Loss tensor(0.0848, grad_fn=<MseLossBackward>)
Loss tensor(0.0846, grad_fn=<MseLossBackward>)
Loss tensor(0.0843, grad_fn=<MseLossBackward>)
Loss tensor(0.0841, grad_fn=<MseLossBackward>)
Loss tensor(0.0839, grad_fn=<MseLossBackward>)
Loss tensor(0.0836, grad_fn=<MseLossBackward>)
Loss tensor(0.0834, grad_fn=<MseLossBackward>)
Loss tensor(0.0831, grad_fn=<MseLossBackward>)
Loss tensor(0.0829, grad_fn=<MseLossBackward>)
Loss tensor(0.0827, grad_fn=<MseLossBackward>)
Loss tensor(0.0824, grad_fn=<MseLossBackward>)
Loss tensor(0.0822, grad_fn=<MseLossBackward>)
Loss tensor(0.0820, grad_fn=<MseLossBackward>)
Loss tensor(0.0818, grad_fn=<MseLossBackward>)
Loss tensor(0.0815, grad_fn=<MseLossBackward>)
Loss tensor(0.0813, grad_fn=<MseLossBackward>)
Loss tensor(0

Loss tensor(0.0513, grad_fn=<MseLossBackward>)
Loss tensor(0.0513, grad_fn=<MseLossBackward>)
Loss tensor(0.0513, grad_fn=<MseLossBackward>)
Loss tensor(0.0512, grad_fn=<MseLossBackward>)
Loss tensor(0.0512, grad_fn=<MseLossBackward>)
Loss tensor(0.0512, grad_fn=<MseLossBackward>)
Loss tensor(0.0511, grad_fn=<MseLossBackward>)
Loss tensor(0.0511, grad_fn=<MseLossBackward>)
Loss tensor(0.0511, grad_fn=<MseLossBackward>)
Loss tensor(0.0510, grad_fn=<MseLossBackward>)
Loss tensor(0.0510, grad_fn=<MseLossBackward>)
Loss tensor(0.0510, grad_fn=<MseLossBackward>)
Loss tensor(0.0509, grad_fn=<MseLossBackward>)
Loss tensor(0.0509, grad_fn=<MseLossBackward>)
Loss tensor(0.0509, grad_fn=<MseLossBackward>)
Loss tensor(0.0508, grad_fn=<MseLossBackward>)
Loss tensor(0.0508, grad_fn=<MseLossBackward>)
Loss tensor(0.0508, grad_fn=<MseLossBackward>)
Loss tensor(0.0507, grad_fn=<MseLossBackward>)
Loss tensor(0.0507, grad_fn=<MseLossBackward>)
Loss tensor(0.0507, grad_fn=<MseLossBackward>)
Loss tensor(0

Loss tensor(0.0464, grad_fn=<MseLossBackward>)
Loss tensor(0.0464, grad_fn=<MseLossBackward>)
Loss tensor(0.0464, grad_fn=<MseLossBackward>)
Loss tensor(0.0464, grad_fn=<MseLossBackward>)
Loss tensor(0.0464, grad_fn=<MseLossBackward>)
Loss tensor(0.0463, grad_fn=<MseLossBackward>)
Loss tensor(0.0463, grad_fn=<MseLossBackward>)
Loss tensor(0.0463, grad_fn=<MseLossBackward>)
Loss tensor(0.0463, grad_fn=<MseLossBackward>)
Loss tensor(0.0463, grad_fn=<MseLossBackward>)
Loss tensor(0.0462, grad_fn=<MseLossBackward>)
Loss tensor(0.0462, grad_fn=<MseLossBackward>)
Loss tensor(0.0462, grad_fn=<MseLossBackward>)
Loss tensor(0.0462, grad_fn=<MseLossBackward>)
Loss tensor(0.0462, grad_fn=<MseLossBackward>)
Loss tensor(0.0461, grad_fn=<MseLossBackward>)
Loss tensor(0.0461, grad_fn=<MseLossBackward>)
Loss tensor(0.0461, grad_fn=<MseLossBackward>)
Loss tensor(0.0461, grad_fn=<MseLossBackward>)
Loss tensor(0.0461, grad_fn=<MseLossBackward>)
Loss tensor(0.0460, grad_fn=<MseLossBackward>)
Loss tensor(0

Loss tensor(0.0434, grad_fn=<MseLossBackward>)
Loss tensor(0.0434, grad_fn=<MseLossBackward>)
Loss tensor(0.0434, grad_fn=<MseLossBackward>)
Loss tensor(0.0434, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0433, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0432, grad_fn=<MseLossBackward>)
Loss tensor(0.0431, grad_fn=<MseLossBackward>)
Loss tensor(0.0431, grad_fn=<MseLossBackward>)
Loss tensor(0

Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0397, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0.0396, grad_fn=<MseLossBackward>)
Loss tensor(0

Loss tensor(0.0385, grad_fn=<MseLossBackward>)
Loss tensor(0.0385, grad_fn=<MseLossBackward>)
Loss tensor(0.0385, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0384, grad_fn=<MseLossBackward>)
Loss tensor(0.0383, grad_fn=<MseLossBackward>)
Loss tensor(0.0383, grad_fn=<MseLossBackward>)
Loss tensor(0.0383, grad_fn=<MseLossBackward>)
Loss tensor(0

In [396]:
# UNQ_C4 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def predict_tweet(tweet, freqs):
    '''
    Input: 
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output: 
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # extract the features of the tweet and store it into x
    x = extract_features(tweet,freqs)
    
    # make the prediction using x and theta
    y_pred = net(torch.from_numpy(x).float())
    
    ### END CODE HERE ###
    
    return y_pred

In [397]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs)))

I am happy -> 0.533605
I am bad -> 0.492973
this movie should have been great. -> 0.528664
great -> 0.527979
great great -> 0.555782
great great great -> 0.583241
great great great great -> 0.610193


In [398]:
# UNQ_C5 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
def test_logistic_regression(test_x, test_y, freqs):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    
    ### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###
    
    # the list for storing predictions
    y_hat = []
    
    for tweet in test_x:
        # get the label prediction for the tweet 
        y_pred = predict_tweet(tweet, freqs)
        
        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1.0)
        else:
            # append 0 to the list
            y_hat.append(0.0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    y_hat = np.array(y_hat) 
    y_hat = y_hat.reshape(test_y.shape[0],1)
    accuracy = ((test_y==y_hat).sum())/test_y.shape[0]

    ### END CODE HERE ###
    
    return accuracy

In [399]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.9945


In [400]:
# Feel free to check the sentiment of your own tweet below
my_tweet = 'love love love love '
predict_tweet(my_tweet, freqs)

tensor([[0.7004]], grad_fn=<SigmoidBackward>)

In [401]:
# Feel free to change the tweet below
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
tensor([[0.4765]], grad_fn=<SigmoidBackward>)
Negative sentiment
